In [ ]:
import os

os.chdir(os.path.dirname(os.getcwd()))

In [ ]:
import os
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import gc
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras import callbacks, optimizers, losses

from IPython import display
from time import sleep

seed = 42
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
autotune = tf.data.AUTOTUNE

# check if tf detect GPU
print("Num GPUs Available: ", len(tf.config.list_physical_devices("GPU")))
print("tensorflow version:", tf.__version__)
print("keras version", keras.__version__)

models_name = [
    "vgg16",
    "resnet50v2",
    "mobilenetv2_1.00_128",
    "inception_v3",
    "convnext_tiny",
]

In [ ]:
from models.victim import build_model

In [ ]:
def get_data(path, label_mode, batch_size, image_size, shuffle=False):
    return image_dataset_from_directory(
        directory=path,
        label_mode=label_mode,
        batch_size=batch_size,
        image_size=image_size,
        shuffle=shuffle,
        smart_resize=True,
    )

In [ ]:
def get_compiled_model(model_name, image_shape):
    model = build_model(model_name, image_shape)
    model.compile(
        optimizer=optimizers.Adam(learning_rate=1e-4),
        loss=losses.BinaryCrossentropy(),
        metrics=["accuracy"],
    )

    return model

In [ ]:
# test load model
for model_name in models_name:
    model = get_compiled_model(model_name, (128, 128, 3))
    model.summary()
    keras.backend.clear_session()
    display.clear_output(wait=True)
    sleep(5)

display.clear_output()

In [ ]:
train_path = ""
valid_path = ""

model_path = ""

objec_path = ""

In [ ]:
epochs = 300
batch_size = 64
image_size = (128, 128)
image_shape = image_size + (3,)

for model_name in models_name:
    keras.backend.clear_session()
    display.clear_output(wait=True)

    train = get_data(train_path, "binary", batch_size, image_size, shuffle=True)
    valid = get_data(valid_path, "binary", batch_size, image_size, shuffle=True)
    train = train.prefetch(buffer_size=autotune)
    valid = valid.prefetch(buffer_size=autotune)

    # training
    model = get_compiled_model(model_name, image_shape)
    print()
    print(f"start training -> {model.name}")
    train_history = model.fit(
        train,
        epochs=epochs,
        validation_data=valid,
        callbacks=[callbacks.EarlyStopping(patience=16, restore_best_weights=True)],
    )

    # fine tune
    print()
    print("start fine tuning...")
    model_file_name = os.path.join(model_path, f"fine_tuned_{model_name}")
    model.get_layer(model_name).trainable = True
    model.compile(
        optimizer=optimizers.Adam(learning_rate=1e-5),
        loss=losses.BinaryCrossentropy(),
        metrics=["accuracy"],
    )
    fine_tune_epochs = 16
    fine_tune_history = model.fit(
        train,
        epochs=fine_tune_epochs,
        validation_data=valid,
        callbacks=[
            callbacks.ModelCheckpoint(filepath=model_file_name, save_best_only=True)
        ],
    )
    print("finish fine tuning!!")

    del train, valid, model
    gc.collect()

display.clear_output()
print("DONE..")